# Python for Climate and Meteorology AMS Online Short Course 2021
## Pangeo Notebook 03: Working with HRRR Data

## In this notebook, we'll cover the following:
1. Access archived HRRR data hosted on AWS in Zarr format
2. Visualize one of the variables (2m temperature) at an analysis time

# <span style="color:purple">0) Preliminaries</span>

In [1]:
import xarray as xr
import s3fs
import metpy
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# <span style="color:purple">1) Access archived HRRR data hosted on AWS in Zarr format</span>

#### A limited set of HRRR grids are stored in Zarr format. A quirk in how these grids were converted from GRIB2 to Zarr means that the dimension variables are defined one directory up from where the data variables are. Thus, our strategy is to use Xarray's `open_mfdataset` method and pass in two AWS S3 file references to these two corresponding directories. 

To interactively browse the contents of this archive, check out this link: [HRRRZarr File Browser on AWS](https://hrrrzarr.s3.amazonaws.com/index.html)

In [2]:
# Access HRRR from AWS ... projection dimensions are in url2
date = '20210216'
hour = '12'
var = 'TMP'
level = '2m_above_ground'
fs = s3fs.S3FileSystem(anon=True)

url1 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var + '/' + level
url2 = 's3://hrrrzarr/sfc/' + date + '/' + date + '_' + hour + 'z_anl.zarr/' + level + '/' + var

print (url1)

s3://hrrrzarr/sfc/20210216/20210216_12z_anl.zarr/2m_above_ground/TMP/2m_above_ground


In [ ]:
file1 = s3fs.S3Map(url1, s3=fs)
file2 = s3fs.S3Map(url2, s3=fs)

ds = xr.open_mfdataset([file1,file2], engine='zarr')

#### Examine the dataset.

In [ ]:
ds

#### The projection information for the HRRR was not readily found in the Zarr representation, so we will define it explicitly here.

#### HRRR Grid Navigation: 
     PROJECTION:          LCC                 
     ANGLES:                38.5   -97.5    38.5
     GRID SIZE:             1799    1059
     LL CORNER:            21.1381 -122.7195
     UR CORNER:            47.8422  -60.9168

In [ ]:
lon1 = -97.5
lat1 = 38.5
slat = 38.5

projData= ccrs.LambertConformal(central_longitude=lon1,
                             central_latitude=lat1,
                             standard_parallels=[slat])

#### Examine the dataset's coordinate variables

In [ ]:
ds.coords

#### Create an object pointing to the dataset's data variable.

In [ ]:
airTemp = ds.TMP

#### Examine the data variable. It is a Dask array.

In [ ]:
airTemp

#### Let's use MetPy to convert the units to Celsius.

In [ ]:
airTemp = airTemp.metpy.convert_units('degC')

#### Verify that the object has the unit change

In [ ]:
airTemp

#### As a result of how we created the dataset, the x- and y- dimensions are attached to the variable. We define objects pointing to them now, so we can pass them to the plotting functions.

In [ ]:
x = airTemp.projection_x_coordinate
y = airTemp.projection_y_coordinate

# <span style="color:purple">2) Visualize one of the variables (2m temperature) at an analysis time </span>

#### First, just use Xarray's `plot` function to get a quick look to verify that things look right.

In [ ]:
airTemp.plot(figsize=(11,8.5))

### Exercise: Before we plot the map, obtain the min and max values from this `DataArray`.
#### Hint: Since the DataArray is actually a Dask array, applying the min and max functions won't actually do the computation. You will need to call Dask's `compute` function to actually trigger the computation.

In [ ]:
# Write your code here

#### Based on the min and max that you obtained, define a range of values used for contouring.

In [ ]:
fint = np.arange(-40,32,2)

#### Now proceed with creating the final graphic. We'll define the plot extent to nicely encompass the HRRR's spatial domain.

In [ ]:
latN = 50.4
latS = 24
lonW = -124
lonE = -71

res = '50m'

fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=projData)
ax.set_extent ([lonW,lonE,latS,latN],crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

# Add the title
tl1 = str('HRRR 2m temperature ($^\circ$C)')
tl2 = str('Analysis valid at: '+ date + hour + ' UTC')
plt.title(tl1+'\n'+tl2,fontsize=16)

# Contour fill
CF = ax.contourf(x,y,airTemp,levels=fint,cmap=plt.get_cmap('coolwarm'))
# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(CF,shrink=0.5)
cbar.set_label(r'2m Temperature ($^\circ$C)', size='large')

## Exercise: try different times and variables.